In [1]:
from GNLP.corpus import Corpus

2020-10-21 13:15:29,593	INFO -- MainProcess corpus.py:17 -- Welcome to the Georgian NLP toolset demo


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
corp = Corpus()
corp.from_file('data/corpuses/kawiki-latest-pages-articles_preprocessed.txt')

{   'counted': 0,
    'prepro_sequence': 0,
    'sequence': 0,
    'stop_words': 373,
    'tokens': 0}

In [4]:
corp.file2sequence(preprocess=True)
corp.count_tokens()

2020-10-21 13:15:29,614	INFO -- MainProcess corpus.py:113 -- ვიწყებ data/corpuses/kawiki-latest-pages-articles_preprocessed.txt-ის დამუშავებას
2020-10-21 13:15:29,709	INFO -- MainProcess corpus.py:126 -- წინადადების რაოდენობა: 74233


In [5]:
corp.count_tokens()

In [6]:
len(corp.counted)

120830

In [7]:
corp.correction("თბილასი")

'თბილისი'

In [8]:
#!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
#!unzip GNLP/data/bert_model/multi_cased_L-12_H-768_A-12.zip

In [9]:
BERT_VOCAB = 'GNLP/multi_cased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'GNLP/multi_cased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'GNLP/multi_cased_L-12_H-768_A-12/bert_config.json'

In [10]:
#!pip install bert-tensorflow 
# conda install tensorflow = 1.14
#!pip install numpy

In [11]:
# !wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

In [12]:
import modeling
import optimization
import run_classifier
import tokenization
import tensorflow as tf
import numpy as np


2020-10-21 13:15:33,927	WARNING -- MainProcess module_wrapper.py:139 -- From /home/tchichua/GeoBERT/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [13]:
import unicodedata

def whitespace_tokenize(text):
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

class BasicTokenizer(object):

    def __init__(self, do_lower_case=True, never_split=None):
        if never_split is None:
            never_split = []
        self.do_lower_case = do_lower_case
        self.never_split = never_split

    def tokenize(self, text, never_split=None):
        never_split = self.never_split + (never_split if never_split is not None else [])
        text = self._clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if token not in never_split:
                if self.do_lower_case:
                    token = token.lower()
                token = self._run_strip_accents(token)
                split_tokens.extend(self._run_split_on_punc(token))
            else:
                split_tokens.append(token)

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text, never_split=None):
        if never_split is not None and text in never_split:
            return [text]
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]
    
    def _clean_text(self, text):
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)
    
def _is_control(char):
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False

def _is_whitespace(char):
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False

def _is_punctuation(char):
    cp = ord(char)
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [14]:
from bert.tokenization import WordpieceTokenizer, load_vocab, convert_by_vocab

class FullTokenizer(object):
    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab = load_vocab(vocab_file)
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case, 
                                              never_split = ['[CLS]', '[MASK]', '[SEP]'])
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)

        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        return convert_by_vocab(self.vocab, tokens)

    def convert_ids_to_tokens(self, ids):
        return convert_by_vocab(self.inv_vocab, ids)

In [15]:
tokenizer = FullTokenizer(vocab_file=BERT_VOCAB, do_lower_case=True)

In [16]:
text = '[CLS] ეს ქუჩვს ვინ დაანაგვიანა [SEP]'
text_mask = text.replace('ქუჩვს', '**mask**')
text_mask

'[CLS] ეს **mask** ვინ დაანაგვიანა [SEP]'

In [17]:
def get_indices(mask, word):
    splitted = mask.split('**mask**')
    left = tokenizer.tokenize(splitted[0])
    middle = tokenizer.tokenize(word)
    right = tokenizer.tokenize(splitted[1])
    indices = [i for i in range(len(left))]
    for i in range(len(right)):
        indices.append(i + len(middle) + len(left))
    
    indices = indices[1:-1]
    tokenized = tokenizer.tokenize(mask.replace('**mask**',word))
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    ids_left = tokenizer.convert_tokens_to_ids(left)
    ids_right = tokenizer.convert_tokens_to_ids(right)
    indices_word = ids_left + ids_right
    return ids, indices, indices_word[1:-1]

In [18]:
possible_states = corp.edit_candidates('ქუჩვს')
possible_states

['ქუჩას', 'ქუჩის']

In [19]:
indices = [get_indices(text_mask, word) for word in possible_states]
ids, seq_ids, word_ids = list(zip(*indices))
ids

([101,
  25579,
  1585,
  32961,
  111518,
  30536,
  1569,
  73701,
  10871,
  93362,
  44970,
  59081,
  19060,
  102],
 [101,
  25579,
  1585,
  32961,
  111518,
  13282,
  1569,
  73701,
  10871,
  93362,
  44970,
  59081,
  19060,
  102])

In [20]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

2020-10-21 13:15:34,331	WARNING -- MainProcess module_wrapper.py:139 -- From /home/tchichua/GeoBERT/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [21]:
class Model:
    def __init__(
        self,
    ):
        self.X = tf.compat.v1.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        embedding = model.get_embedding_table()
        
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, embedding, transpose_b = True)
            self.logits = tf.nn.bias_add(logits, output_bias)

In [22]:
tf.compat.v1.disable_eager_execution()

In [23]:
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')

2020-10-21 13:15:34,403	WARNING -- MainProcess module_wrapper.py:139 -- From /home/tchichua/GeoBERT/modeling.py:171: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

2020-10-21 13:15:34,406	WARNING -- MainProcess module_wrapper.py:139 -- From /home/tchichua/GeoBERT/modeling.py:409: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

2020-10-21 13:15:34,432	WARNING -- MainProcess module_wrapper.py:139 -- From /home/tchichua/GeoBERT/modeling.py:490: The name tf.assert_less_equal is deprecated. Please use tf.compat.v1.assert_less_equal instead.

2020-10-21 13:15:34,446	WARNING -- MainProcess lazy_loader.py:50 -- 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend

In [24]:
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(119547,) dtype=float32_ref>]

In [25]:
saver = tf.train.Saver(var_list = var_lists + cls)
saver.restore(sess, BERT_INIT_CHKPNT)

2020-10-21 13:15:37,634	INFO -- MainProcess saver.py:1284 -- Restoring parameters from multi_cased_L-12_H-768_A-12/bert_model.ckpt


In [26]:
masked_padded = tf.keras.preprocessing.sequence.pad_sequences(ids,padding='post')
masked_padded.shape

(2, 14)

In [27]:
preds = sess.run(tf.nn.softmax(model.logits), feed_dict = {model.X: masked_padded})
preds.shape

(2, 14, 119547)

In [28]:
scores = []

for no, ids in enumerate(seq_ids):
    scores.append(np.prod(preds[no, ids, word_ids[no]]))
    
scores

[0.94074315, 0.9292732]

In [29]:
prob_scores = np.array(scores) / np.sum(scores)
prob_scores

array([0.5030668, 0.4969332], dtype=float32)

In [30]:
probs = list(zip(possible_states, prob_scores))
probs.sort(key = lambda x: x[1])  
probs[::-1]

[('ქუჩას', 0.5030668), ('ქუჩის', 0.4969332)]